# MongoDB.
    Bailey Smith
    November 28 2017

In [1]:
import re
import json
import numpy as np
from pymongo import MongoClient

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that is loaded with `json.loads()`.

Creation of a MongoDB database and initialization of a collection in the database.
Clears existing contents of the collection, then fills the collection one line at a time with the data from `trump.json`.
Checks that collection has 95,643 entries.

In [2]:
client = MongoClient()

# Create a new database.
db = client.db1

# Create a new collection in the db database.
col = db.collection1

col.insert_one({'name': 'Jack', 'age': 23})

col.delete_many({})

In [5]:
with open('trump.json','r') as f:
    for line in f:
        data = json.loads(line) # load json
        col.insert_one(data) # insert each line into database

In [6]:
print(col.count())

95643


Queries of the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [9]:
# Query for the word russia in the text of each tweet
result1 = col.find({'text' : {'$regex' : re.compile('russia', re.IGNORECASE)}})
print("Number of tweets that include the word Russia:",result1.count())

Number of tweets that include the word Russia: 5841


In [10]:
# Query for time_zone that is one of the main US time zones
result2 = col.find({'$or' : [{'user.time_zone' : "Central Time (US & Canada)"}, 
                             {'user.time_zone' : "Pacific Time (US & Canada)"}, 
                             {'user.time_zone' : "Eastern Time (US & Canada)"}, 
                             {'user.time_zone' : "Mountain Time (US & Canada)"}]})
print("Number of tweets that came from one of the main continental US time zones:",result2.count())

Number of tweets that came from one of the main continental US time zones: 33939


In [12]:
languages = col.distinct('lang') # get list of all languages that occur
my_dict = dict([])
for l in languages:
    result = col.find({'lang' : l})
    count = result.count()
    my_dict[l] = count

In [13]:
print("Frequency of languages")
print(my_dict)

Frequency of languages
{'en': 84919, 'fr': 1235, 'es': 3027, 'tr': 588, 'nl': 212, 'pt': 632, 'und': 1653, 'it': 423, 'de': 635, 'da': 35, 'zh': 1, 'ru': 29, 'pl': 108, 'no': 21, 'eu': 6, 'tl': 48, 'in': 121, 'cs': 13, 'ht': 21, 'ro': 21, 'sl': 3, 'sv': 55, 'fi': 20, 'ar': 30, 'et': 23, 'lt': 8, 'hu': 6, 'el': 10, 'ja': 17, 'lv': 2, 'vi': 7, 'fa': 5, 'ko': 2, 'ur': 1, 'cy': 2, 'th': 3, 'is': 1, 'iw': 1, 'uk': 3}


Queries of the first Twitter collection for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [17]:
# Query for followers_count that is >= 1000 then sorting
new_results = col.find({'user.followers_count': {'$exists': True}}).sort('user.followers_count', -1)

In [18]:
s = set()
for user in new_results[:10]:
    s.add(user['user']['name']) # add users to a set so we can get the top 5 unique users
    if len(s) == 5:
        break
print("Usernames of the 5 most popular tweeters:",s)

Usernames of the 5 most popular tweeters: {'Fox News', 'CNN', 'TIME', 'Wall Street Journal', 'Reuters Top News'}


In [19]:
# Query for hashtags that have at least 5 entries and sort based on indices (ie the position of the hashtag in the tweet)
hashtags = col.find({'entities.hashtags.4': {'$exists': True}}).sort('entities.hashtags.4.indices.0')

In [20]:
print("Earliest spot that a 5th hashtag appears:",hashtags[0]['entities']['hashtags'][4]['indices'][0])

Earliest spot that a 5th hashtag appears: 23


In [21]:
# Query for coordinates  and sort based on latitude
coordinates = col.find({'coordinates.coordinates':{'$exists': True}}).sort('coordinates.coordinates.0',-1)

In [22]:
print("Coordinates of the tweet that came from the northernmost location:",coordinates[0]['coordinates']['coordinates'])

Coordinates of the tweet that came from the northernmost location: [28.99237342, 41.06803023]


Cleaned the Twitter collection in the following ways.

- Got rid of the `"retweeted_status"` field in each tweet.
- Updated every tweet from someone with at least 1000 followers to include a popular field whose value is True. Reported the number of popular tweets.

In [75]:
#Get rid of "retweeted_status" field
no_retweet = col.update_many({'retweeted_status' : {'$exists': True}}, {'$unset' : {'retweeted_status' : ""}})

In [26]:
print(col.find_one().keys())

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms', 'popular'])


In [4]:
# Add popular field that is set to True for users with 1000 followers or more
col.update_many({"user.followers_count" : {'$gte' : 1000}}, {'$set': {'popular': True}})

In [6]:
col.find_one({"user.followers_count" : {'$gte' : 1000}})['popular']

True